# Face Analyzer

## setup

First we will import the modules

In [ ]:
import tensorflow as tf
import cv2
from mtcnn import MTCNN
import numpy as np
import math
import os
import logging

MTCNN causes tensorflow warnings so we will want to disable them, in addition tesnorflow needs to be configured

In [ ]:
logging.getLogger('tensorflow').disabled = True

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

Make sure your GPU is active

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## student object
In order to get attentiveness each face that is found creates a student object. inside the object are attributes that I will discuss later

In [ ]:
class Student:
    
    def __init__(self, face, name):
        self.face = face
        self.name = name
        self.box = face['box']
        self.face_points = face['keypoints']
        self.attention_points = (0,0)
        self.attention_angle_list = []
        self.mode_attention_angle = 0
        self.attention_angle_per_frame = []
        self.absent_from_frame = 0

    @property
    def update_face(self):
       self.face = face
       self.box = face['box']
       self.face_points = face['keypoints']

    @update_face.setter
    def update_face(self, face):
        self.face = face
        self.box = face['box']
        self.face_points = face['keypoints']
        
    def __repr__(self):
        return "student('{}', '{}')".format(self.box, self.name)

    def __str__(self):
        return "student: {} attentiveness: {}".format(self.name, self.attention_angle_list)

## Code
in this section I will first show the main `student_attentiveness` function and show the functions it calls on inside it

In [ ]:
def student_attentiveness():
    """gets the student attentiveness for the lecture

    Returns:
        a csv of attentiveness

    """
    if os.name == 'posix':
        delimiter = '/'
    else: 
        delimiter = '\\'
    
    current_directory = os.getcwd()
    data_directory = os.path.abspath(os.path.join(current_directory, os.pardir + delimiter + 'data'))
    screenshot_directory = os.path.abspath(os.path.join(data_directory + delimiter + 'screenshot'))
    list_of_files = sorted(os.listdir(screenshot_directory))
    
    for i in range(len(list_of_files)):
        img = cv2.imread(screenshot_directory + delimiter + list_of_files[i])
        # print(screenshot_directory + delimiter + list_of_files[i])
        if i == 0:
            student_list = initial_frame(img)
        for student in student_list:
            try:
                next_frame = cv2.imread(screenshot_directory + delimiter + list_of_files[i+1])
                find_student_next_frame(student, next_frame)
                check_for_absent(student_list)
            except IndexError:
                break
        find_new_students(student_list, next_frame)
    classroom_angles = []            
    for student in student_list:
        get_mode_angle(student)
        get_attention_per_frame(student)
        classroom_angles.append(student.attention_angle_per_frame)

    # TODO: fix this, its padding because sometimes the length of student attention angles arent the same
    max_len = max(len(x) for x in classroom_angles) 
    for i in classroom_angles:
        if len(i) != max_len:
            i.append(0.6)
    
    
    avg_across_lecture = np.mean(classroom_angles,axis=0)
    abs_avg_lecture = [abs(x) for x in avg_across_lecture]
    np.savetxt(data_directory + delimiter + 'attentiveness.csv', abs_avg_lecture, delimiter=',', header='attentiveness')
    return student_list

It first determines what operating system you have. this is important to decide what delimiter to use. (not sure if this is necessary, I think you can just do this all with the os.path stuff). Then we get the data directory and the screenshot directory, this will be important for reading and writing files.

next is the for loop that iterates through the sampled lecture. it starts by reading the first image and if it is the first iteration it goes and initializes the student with the `initial_frame` function which i will discuss now.

In [ ]:
def initial_frame(img):
    """initializes student object

    Args:
       img: first image in directory

    Returns:
        list of student objects

    """
    student_list = []
    img_size = img.shape
    faces = find_faces(img)
    for i in range(len(faces)):
        student_list.append(Student(faces[i],str(i)))

    for student in student_list:
        get_pose_direction(student,img)
        get_angle(student)
    return student_list

Initial_frame takes an image and returns a list of student objects. first it initializes the list gets the image size and then finds the faces in the image using the `find_faces` function which we will look at right now and then come back to this. 

In [ ]:
def find_faces(img):
    """
    Find the faces in an image
    
    Args:
        img : Image to find faces from
   
    Returns:
        faces : list of dictionaries of faces with keypoints

    """
    min_conf = 0.9
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pixels = np.asarray(img)

    detector = MTCNN()

    detected = detector.detect_faces(pixels)
    faces = [i for i in detected if i['confidence'] >= min_conf]
    return faces


This function takes an image and returns a list of faces, it only gets the faces that have above 90% confidence. Now lets go back to the `initial_frame` function

After we have the list of faces it starts to iterate through the faces assigning them to a student object with i being the student name. (student: 1, student: 2, etc.) after that it iterates through the student list and gets the pose and angle of the student by using the `get_pose_direction` and `get_angle` respectively.

In [ ]:
def get_pose_direction(student,im):
    img_size = im.shape
    focal_length = img_size[1]
    center = (img_size[1]/2, img_size[0]/2)
    camera_matrix = np.array(
        [[focal_length, 0, center[0]],
         [0, focal_length, center[1]],
         [0, 0, 1]], dtype = "double"
    )
    model_points = np.array([
        (0.0, 0.0, 0.0),             # Nose tip
        (-225.0, 170.0, -135.0),     # Left eye left corner
        (225.0, 170.0, -135.0),      # Right eye right corne
        (-150.0, -150.0, -125.0),    # Left Mouth corner
        (150.0, -150.0, -125.0)      # Right mouth corner
    ])
    dist_coeffs = np.zeros((4,1))
    image_points = np.array([
        student.face_points['nose'],        
        student.face_points['left_eye'],    
        student.face_points['right_eye'],   
        student.face_points['mouth_left'],  
        student.face_points['mouth_right']  
    ], dtype="double")
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_UPNP)
    
    (nose_end_point2D, jacobian) = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)

    # for p in image_points:
    #     cv2.circle(im, (int(p[0]), int(p[1])), 3, (0, 0, 255), -1)
      
    student.attention_points = ((int(image_points[0][0]), int(image_points[0][1])),(int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1])))

I found this algorithm [here](https://learnopencv.com/head-pose-estimation-using-opencv-and-dlib/) which does a better job explaining it than I do. it uses the keypoints on a face to determine where the student is facing. This function then returns two points, one at he tip of the nose and the other in the direction of where the student is facing. ![example](../Image/pose.png)

now lets take a look at the `get_angle` function from the `initial_frame` function

In [ ]:
def get_angle(student):
    """gets the angle of the slope of the points (best we could do), appends that angle to angle list

    Args:
       student: thestudent working on
    """
    attention_points = student.attention_points
    try:
        m = ((attention_points[1][1] - attention_points[0][1])/(attention_points[1][0] - attention_points[0][0]))
        angle = int(math.degrees(math.atan(m)))
        student.attention_angle_list.append(angle)
    except ZeroDivisionError:
        angle = -90
        student.attention_angle_list.append(angle)

This takes in two points found from the get_pose function and determines the angle. it then appends the angle to the student object. this will be used to analyze the attentiveness per frame. after this is called in the `initial_frame` function that function is done and returns the student_list to the `student_attentiveness` function which we will now go back to.

After it initializes students it then iterates through the students list. it then starts to analyze the next frame in the lecture. from there it runs the `find_student_next_frame` function

In [ ]:
def find_student_next_frame(student,next_image):
    """finds student in next frame and updates student attributes

    Args:
       student: student object 
       next_image: image for next frame

    """
    top_left_point = (student.face_points['left_eye'][0], student.face_points['left_eye'][1])
    bottom_right_point = (student.face_points['mouth_right'][0], student.face_points['mouth_right'][1])
    
    student_box = extend_box(top_left_point, bottom_right_point, 50)
    
    mask = np.zeros(next_image.shape[:2], dtype=np.uint8)
    mask[student_box[0][1]:student_box[1][1]+1,student_box[0][0]:student_box[1][0]+1] = 255
    rect_img = cv2.bitwise_and(next_image,next_image,mask=mask)
    
    face = find_faces(rect_img)
    if len(face) > 1:
        # TODO(#17): find a way to decrease dx and call function again
        #print("more faces")
        
        # TODO: add noise here instead of zero
        student.attention_angle_list.append(0)
    if len(face) < 1:
        #if no faceappend with noise
        #print("no face")
        student.attention_angle_list.append(0)
        student.absent_from_frame += 1
            
    if len(face) == 1:
        student.update_face = face[0]
        get_pose_direction(student,next_image)
        get_angle(student)
    
    # cv2.imshow("focused student", rect_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    #return student

This takes a student object and a next image, it cuts down the image to only a region where the student was scene in the last frame and then finds the face from that cropped image to see if the student is still there. it creates a region to search for faces using the `extend_box` function (we will look at after this). the image is cropped using the extended region and then searches for faces in that region. if there is more than one face then we append the student attention angle list with noise. if there is no face we say the student is absent from the frame and increment by 1 (if it reaches 10 then the student is removed from the list, this is to take into account a student moving from one side of the classroom to the other) lastly if the face is exactly 1 then we update the pose and student angle.
![image](../Image/cropped.png)

In [ ]:
def extend_box(top_left,bottom_right,dx):
    """gets a box from a face in order to crop image to look for face again

    Args:
       top_left:     point for top left
       bottom_right: point for bottom_right
       dx:           the new box size

    Returns:


    """
    top_left = (top_left[0] - dx, top_left[1] - dx)
    bottom_right = ((bottom_right[0] + dx, bottom_right[1] + dx))
    return (top_left, bottom_right)

the arguments this takes in is the left eye and the mouth right. it extends the box by 50 pixels. and returns a new region to scan for faces.

now lets go back to `student_attentiveness`, after it checks for student in next frame. it then checks if it was absent 10 times (I can see that reseting the counter might be important if the student is found in again) using the `check_for_absent` function

In [ ]:

def check_for_absent(student_list):
    """checks if student is missing for 10 frames and removes them

    Args:
       student_list: list of students

    Returns:
        list of students

    """
    i = 0
    for student in student_list:
        if student.absent_from_frame >= 10:
            student_list.pop(i)
            #print("removed student")
        i+=1

this takes in a student list. If the student in the student list has been absent from the region it is searching for in the `find_student_next_frame` for more than 10 times. it is safe to say that the student was either walking to their seat or has left the class room. in both of these cases it is good to remove them in order to get attentiveness score. Now lets go back to `student_attentiveness`

after all the known students have been analyzed, it is time to search for new students, this is to make sure the program didn't miss any or for students that arrive late. the students that arrive late will have their attention_angle_list filled with noise as they were not paying attention previously. it will do this with the `find_new_students` function

In [ ]:
def find_new_students(student_list, next_frame):
    """looks for new students and appends them to student_list

    Args:
       student_list: list of objects
       next_frame: next image

    Returns:
        student_list

    """
    faces = find_faces(next_frame)
    for face in faces:
        
        found = False
        top_left = face['keypoints']['left_eye']
        bottom_right = face['keypoints']['mouth_right']
        extended_top_left, extended_bottom_right = extend_box(top_left, bottom_right, 50)
        
        
        for student in student_list:
            img = next_frame.copy()
            test_top_left = student.face_points['left_eye']
            test_bottom_right = student.face_points['mouth_right']
            
            # cv2.circle(img, test_top_left,1,(0,0,255),2)
            # cv2.circle(img, test_bottom_right,1,(0,0,255),2)
            # cv2.rectangle(img, extended_top_left, extended_bottom_right, (255,255,0),1)
            # cv2.imshow("newframe", img)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            
            if check_box(extended_top_left,extended_bottom_right,test_top_left,test_bottom_right):
                found = True
                #print("found")
                break
            
    
        if found == False:
            #print("added student")
            max_name = max([int(x.name) + 1 for x in student_list])
            # TODO: instead of zero add noise
            attention_list = [0 for i in range(len(student_list[0].attention_angle_list))]
            student_list.append(Student(face,max_name))
            student_list[-1].attention_angle_list = attention_list

I feel like I can change this somehow. right now what the function iterates through the known students finds the ones it already knows and when it finds it, the loop just breaks. ive tried updating the student attributes when it finds the students from here before. but the issue is that it searches through the same image so many times that it would update the students a lot and produce results that are inaccurate. 

anyways i think that showing what this is doing through pictures is best. first it finds all the faces in the image. then it does the extend box thing that i talked about before. then it iterates through the student list and places the two points. if the two points are inside the box it knows that the student is already accounted for. otherwise it adds a new student. 

This student will be found. this first shows that face points are not in the box so the student is not found yet. the second picture the points are in the box so the student is found and will then move onto the next face.
![not_found](../Image/student_not_found.png)
![found](../Image/student_found.png)

This next image shows a student that showed up late. they were not in the last frame but they are in the current one. they will be added to the student list.
![will_not_be_found](../Image/will_not_be_found.png)

There was a function in `find_new_student` that i havent talked about yet, it is the `check_box` function

In [ ]:
def check_box(rect_top_left, rect_bottom_right, test_top_left, test_bottom_right):
    """checks if the face points are in the boxx

    Args:
       rect_top_left: box top left point
       rect_bottom_right: box bottom right
       test_top_left: student left eye 
       test_bottom_right: student mouth right

    Returns:
        bool if points are in box

    """
    if (rect_top_left[0] < test_top_left[0]) and (rect_top_left[0] < test_bottom_right[0]) \
    and (rect_bottom_right[0] > test_top_left[0]) and (rect_bottom_right[0] > test_bottom_right[0]) \
    and (rect_top_left[1] < test_top_left[1]) and (rect_top_left[1] < test_bottom_right[1]) \
    and (rect_bottom_right[1] > test_top_left[1]) and (rect_bottom_right[1] > test_bottom_right[1]):
        return True
    return False

this is pretty simple. the coordinates for images increase from left to right and increase from top to bottom. so in order to find if points are within a the parameters of the box the test are compared with against each other. \[0\] is the x position and \[1\] is the y position. 

now we go back to `student_attentiveness`. once it analyzes all the files it iterates through the student list and does calculations on the attention angle. we (the group) made an assumption here, students are paying attention the majority of the time. so the program finds the mode angle. in order to do this the angles need to be rounded. 

In [ ]:
def get_mode_angle(student):
    """gets the mode angle (students are assumed to be paying attention most of the time)

    Args:
       student: student object


    """
    angle_list = student.attention_angle_list
    
    # filter out value for zero
    for i in range(len(angle_list)):
        if abs(angle_list[i]) <= 2:
            if angle_list[i] > 0:
                angle_list[i] = 3
            else:
                angle_list[i] = -3
            
    binned = [5 * round(x/5) for x in angle_list]
    mode = max(set(binned), key=binned.count)
    student.mode_attention_angle = mode

i dont want the mode angle to be 0 so i made it so that it will always round either to 5 or negative 5. now back to `student_attentiveness`

the next function call is `get_attention_per_frame` which really just divides the attention at that frame by the mode angle to get a ratio of attention.

In [ ]:
def get_attention_per_frame(student):
    """gets the ratio of the attention at frame vs mode_attention_angle

    Args:
       student: student object


    """
    attention_list = student.attention_angle_list
    mode_attention_angle = student.mode_attention_angle
    student.attention_angle_per_frame = [x/mode_attention_angle for x in attention_list]

now back to `student_attentiveness` the last thing it does is pad the classroom angles. This is a bug in my code. somewhere it doesn't fill the attention_list all the way. all the ones that arent the same are off by one. i dont know where this is. so i need help finding so i dont have to pad it. lastly it gets the mean of the attention angles per frame for the whole classroom and that is what we are using for our attentiveness. I think this program is pretty much done i dont think i will change it too much in the future. I might fill the attention with actual noise instead of zero.

In [ ]:
if __name__ == '__main__':
    from split_video import split
    from analyze_video import screencap_video

    # lecture = 'class1facingstudents.mov'
    # split(lecture, 'students')
    
    # screencap_file = 'students-output-video.mp4'
    # screencap_video(screencap_file)

    student_list = student_attentiveness()